In [11]:
import pandas as pd
from dict import depression_labels, newsgroups_labels, emotion_labels, news_labels, yahoo_labels, rating_labels
import os
import sys

In [12]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [13]:
dataset = '20_newsgroups'
path = 'extracted/results/cot_random_samples_cohere_temp0/llama3'
new_path = 'extracted_new' + path[9:]


In [14]:
new_path

'extracted_new/results/cot_random_samples_cohere_temp0/llama3'

In [15]:
def get_dict(dataset):
    if(dataset == 'ag_news'):
        return news_labels
    if(dataset == 'yahoo'):
        return yahoo_labels
    if(dataset == '20_newsgroups'):
        return newsgroups_labels
    if(dataset == 'social_media'):
        return depression_labels
    if(dataset == 'go_emotions'):
        return emotion_labels
    if(dataset == 'sst5'):
        return rating_labels
    

In [16]:
labels = get_dict(dataset)

In [17]:
list(labels.keys())

['rec.sport.hockey',
 'soc.religion.christian',
 'rec.sport.baseball',
 'rec.motorcycles',
 'sci.crypt',
 'rec.autos',
 'sci.med',
 'sci.space',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'sci.electronics',
 'comp.windows.x',
 'comp.graphics',
 'misc.forsale',
 'comp.sys.mac.hardware',
 'talk.politics.mideast',
 'talk.politics.guns',
 'alt.atheism',
 'talk.politics.misc',
 'talk.religion.misc']

In [18]:
dataset

'20_newsgroups'

In [19]:
import os
import pandas as pd
import re

def process_csv_files(folder_path, prefix, labels, save_path):
    """
    Przetwarza pliki CSV w folderze, które zaczynają się od podanego prefixu.
    Z kolumny 'output' wyodrębnia ostatnią wartość w nawiasach kwadratowych [ ],
    zapisuje ją do 'extracted_label'. Następnie porównuje z listą `labels` i
    jeśli nie pasuje, kopiuje wartość z 'extracted_label_by_llm' do
    'extraction_complete', a jeśli pasuje, zostawia wartość z 'extracted_label'.

    :param folder_path: Ścieżka do folderu z plikami CSV
    :param prefix: Prefiks, wg którego filtrowane są pliki
    :param labels: Lista poprawnych labeli
    :param save_path: Folder, do którego zapisywane są przetworzone pliki
    """
    # Upewnij się, że ścieżka docelowa istnieje
    os.makedirs(save_path, exist_ok=True)
    
    def extract_last_square(text):
        matches = re.findall(r'\[([^\[\]]+?)\]', str(text))
        return matches[-1] if matches else None

    for filename in os.listdir(folder_path):
        if filename.startswith(prefix) and filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)

            # Wyciągnięcie etykiety z nawiasów kwadratowych
            df['extracted_label'] = df['output'].apply(extract_last_square)

            # Porównanie z poprawnymi labelami
            def determine_extraction_complete(row):
                return row['extracted_label_by_llm'] if str(row['extracted_label']) not in labels else row['extracted_label']

            df['extraction_complete'] = df.apply(determine_extraction_complete, axis=1)
            

            # Zapis do pliku z oryginalną nazwą + _processed.csv
            output_filename = os.path.splitext(filename)[0] + '.csv'
            print(output_filename)
            output_path = os.path.join(save_path, output_filename)
            df.to_csv(output_path, index=False)
            print(f"Zapisano przetworzony plik: {output_path}")


In [20]:
process_csv_files(path, dataset, list(labels.keys()), new_path )

20_newsgroups_llama3_cot_random_samples_cohere_llama3_random42_80_temp0_exp1.csv
Zapisano przetworzony plik: extracted_new/results/cot_random_samples_cohere_temp0/llama3/20_newsgroups_llama3_cot_random_samples_cohere_llama3_random42_80_temp0_exp1.csv
